In [1]:
%pwd 
%cd /workspaces/tensorflow-wavelets
import OpenDVC
import numpy as np
import load
import tensorflow as tf
import matplotlib.pyplot as plt

/workspaces/tensorflow-wavelets


2022-01-31 19:34:26.002668: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [ ]:
batch_size = 1
EPOCHS = 15
Height = 240
Width = 240
Channel = 3
lmbda = 256
lr_init = 1e-4
samples = 1200
I_QP=27

args = OpenDVC.Arguments()

last_stop = 164
for i in range(1000):
    print("Iteration number", i)
    checkponts_last_path = "checkpoints_testME4_no_norm{}/".format(last_stop+i)
    checkponts_new_path = "checkpoints_testME4_no_norm{}/".format(last_stop+i+1)

    model = OpenDVC.OpenDVC(width=Width, height=Height, batch_size=batch_size, num_filters=128, lmbda=lmbda)
    # model.summary()
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_init),
    )
    print("* [Model compiled]...")

    print("* [Loading dataset]...")
    data = load.load_data_vimeo90k("/mnt/WindowsDev/Developer/tensorflow-wavelets/folder_cloud.npy", samples, Height, Width, Channel, I_QP)

    dataset = tf.data.Dataset.from_tensor_slices(data)
    print("Loading weights")
    
    model.load_weights(checkponts_last_path)
    print("Going to fit")

    hist = model.fit(
            dataset,
            epochs=EPOCHS, 
            verbose=1, 
            callbacks=
                [
                OpenDVC.MemoryCallback(),
                tf.keras.callbacks.ModelCheckpoint(filepath=checkponts_new_path, save_weights_only=True, save_freq='epoch',monitor="loss",mode='min',  save_best_only=True, verbose=1), 
                tf.keras.callbacks.TerminateOnNaN(),
                tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3),
                tf.keras.callbacks.TensorBoard(log_dir=args.backup_restore, histogram_freq=0, update_freq="epoch"),            
                ],

            )  

    path = load.load_random_path("/mnt/WindowsDev/Developer/tensorflow-wavelets/folder_cloud.npy")
    out_bin = "/workspaces/tensorflow-wavelets/Development/OpenDVC/Test_com/test{}.bin".format(i)
    out_decom = "/workspaces/tensorflow-wavelets/Development/OpenDVC/Test_com/testdcom{}.png".format(i)
    p_on_test = "/workspaces/tensorflow-wavelets/Development/OpenDVC/Test_com/test_p_frame{}.png".format(i)
    i_on_test = "/workspaces/tensorflow-wavelets/Development/OpenDVC/Test_com/test_i_frame{}.png".format(i)

    i_frame = path + 'im1' + '.png'
    p_frame = path + 'im2' + '.png'
    print(i_frame)

    OpenDVC.write_png(p_on_test, OpenDVC.read_png_crop(p_frame, 240, 240))
    OpenDVC.write_png(i_on_test, OpenDVC.read_png_crop(i_frame, 240, 240))

    OpenDVC.compress(model, i_frame, p_frame, out_bin, 240, 240)
    OpenDVC.decompress(model, i_frame, out_bin, out_decom, 240, 240)
